In [49]:
# !pip install gensim
# !pip install nltk
# !pip install sklearn

In [50]:
import pandas as pd
import nltk
import itertools
import numpy as np
import re        
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim import corpora, models
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk import sent_tokenize, word_tokenize
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer


In [51]:
pd.options.display.max_colwidth = 100

In [52]:
base = pd.read_csv("../dados/nlp/baselimpa.csv")
# Not included in the repo due to privacy concerns

In [53]:
base[base['tweet'].str.contains("escola")]['tweet']

7                  estudantes da escola municipal do complexo da maré agachados sob ataque do caveirão aéreo
10       #repost @bertjoi with get_repost ・・・ agatha. 8 anos. tiro nas costas. estava dentro da kombi/lot...
12       #witzelassassino.  parem de nos matar!  abaixo criançasda escolamunicipal no pinheiro, complexo ...
13       nas escolas que foram alvos do caveirão aéreo, funcionários ainda estão com medo. a maioria se m...
17       nós, moradores minimamente alertamos em dias de operações policiais: atividade morador, não leve...
                                                        ...                                                 
10347    aguia de ouro tentou dar uma lacradinha no ensaio técnico dessa semana (sexta se não me engano) ...
10348    a crítica política da águia de ouro é mais um tiro no pé dessa escola de samba que logo logo vai...
10463                                                        eu dá escola hoje vi os cana joga tiro do águia
10583    águia deu 

In [54]:
def tokenization_s(sentences): 
    s_new = []
    for sent in (sentences[:][0]): 
        s_token = sent_tokenize(sent)
        if s_token != '':
            s_new.append(s_token)
    return s_new

In [55]:
def preprocess(text):
    clean_data = []
    for x in (text[:][0]): #this is Df_pd for Df_np (text[:])
       # new_text = re.sub('<.*?>', '', x)   # remove HTML tags
        new_text = re.sub(r'[^\w\s]', '', x) # remove punc.
       # new_text = re.sub(r'\d+','',new_text)# remove numbers
        new_text = new_text.lower() # lower case, .upper() for upper          
        if new_text != '':
            clean_data.append(new_text)
    return clean_data

In [56]:
def tokenization_w(words):
    w_new = []
    for w in (words[:][0]):  # for NumPy = words[:]
        w_token = word_tokenize(w)
        if w_token != '':
            w_new.append(w_token)
    return w_new

In [57]:
snowball = SnowballStemmer(language = 'portuguese')
def stemming(words):
    new = []
    stem_words = [snowball.stem(x) for x in (words[:][0])]
    new.append(stem_words)
    return new

In [58]:
msgs = base['tweet'].astype(str)

In [59]:
#nltk.word_tokenize(base.tweet)

In [60]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords_en = nltk.corpus.stopwords.words('english')
# Define palavras a serem excluídas
newStopWords = ['lula','http','3r6nsl','u200d','helicoptero','policia',
                'aeronave','ly','tb','tche', 'pra', '…','cv43sn','gate21','xa0','tinyurl',
                'helicóptero','durante','aguia','bit','https','fu','ahh','br','tiro',
                'bit','favela','sido','cara','cv43sn\'','enquanto', 'pic', 'twitter', 'www', 'ser'
               'ver','suzano','tipo','xa0\'','bom','bolivia','anti','diz','xa0helicoptero','apos','dois',
                'list','rio','da','em','quem','por','na','mas','ja','era','para','mais','se','nao','do',
                'que','de','ou','com','teria','sempre','outro','ao','os','duas','at','uma','um',
                'tiros','tiro','disparo','helicoptero','q','ta','pq','html','01','2011','tá','ig_twitter_share','uns']

# STOPWORDS = nltk.corpus.stopwords.words('portuguese')

stopwords.extend(newStopWords)
stopwords.extend(stopwords_en)
len(stopwords)

469

In [61]:
# stopwords

In [62]:
# Topic model w/ LDA

tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words= stopwords)
doc_term_matrix = tfidf_vect.fit_transform(base.tweet.values.astype('U'))

nmf = NMF(n_components=3, random_state=42)
nmf.fit(doc_term_matrix )

for i,topic in enumerate(nmf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-30:]])
    print('\n')

/home/abitporu/.pyenv/versions/3.9.0/envs/lede/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


Top 10 words for topic #0:
['proteja', 'alto', 'dia', 'cana', 'porra', 'gente', 'voando', 'tava', 'tão', 'dessa', 'morro', 'mané', 'hora', 'nada', 'acordei', 'caralho', 'baixinho', 'mano', 'rasante', 'crlh', 'passou', 'mt', 'vários', 'deus', 'crl', 'passando', 'cima', 'baixo', 'águia', 'dando']


Top 10 words for topic #1:
['moradores', 'tudo', 'dentro', 'ter', 'rj', 'atirando', 'manhã', 'perto', 'deu', 'civil', 'complexo', 'rua', 'pm', 'hoje', 'nada', 'governador', 'lá', 'ser', 'gente', 'escola', 'agora', 'status', 'cima', 'dar', 'operação', 'sobrevoando', 'vai', 'casa', 'polícia', 'aqui']


Top 10 words for topic #2:
['ouvir', 'hj', 'acordada', 'hoje', 'agr', 'águia', 'acordo', 'manhã', 'ouvindo', 'escutei', 'sobrevoando', 'nada', 'ouvi', 'dormir', 'susto', 'despertador', 'acorda', 'acordou', 'bala', 'morador', 'acerta', 'som', 'perdida', 'terror', 'botando', 'chega', 'acordar', 'dia', 'acordei', 'barulho']




In [47]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=5, random_state=42)

In [64]:
# from sklearn.feature_extraction.text import CountVectorizer

# count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words= stopwords)
# doc_term_matrix = count_vect.fit_ransform(base.tweet.values.astype('U'))

# doc_term_matrix

AttributeError: 'CountVectorizer' object has no attribute 'fit_ransform'

In [48]:
# Remove palavras comuns do dicionário padrão
tweets = [[word for word in tweet_words if not word in stopwords]
              for tweet_words in msgs]

In [32]:
words_in_tweet = [tweet.lower().split() for tweet in base.tweet]

In [34]:
# words_in_tweet

In [18]:
# Conta palavras mais frequentes
#palavras = list(itertools.chain(*base.tweet))

#counts_no_urls = collections.Counter(palavras)

#counts_no_urls.most_common(200)

In [ ]:
# Mostra as mensagens mais retweetadas
#pd.set_option('display.max_colwidth', -1)

#base.nlargest(50, columns='retweets_count')[['date','tweet', 'username','retweets_count']]

In [ ]:
#corpus = [dictionary.doc2bow(mensagens) for text in mensagens]